In [1]:
import nltk
import string
import numpy as np
import pandas as pd
import os
from nltk.corpus import stopwords


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('timesofindia_train.csv', sep='\t')

In [12]:
def word_embeddings(path):
    embeddings_index = dict()
    f = open(path)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [13]:
glove_300d = word_embeddings("/home/user/Desktop/timesofindiatrainfiles/glove.6B/glove.6B.300d.txt")

In [14]:
df_sent = (df['sentence'].str.len() > 10)
ne_df = df.loc[df_sent]
df = ne_df
df = df[~df.sentence.str.startswith('https')]
df = df[~df.sentence.str.startswith('url : ')]

In [15]:
sent_list = []
label_list = []

sent_list = df["sentence"].tolist()
label_list = df["etype"].tolist()

print(len(label_list))
print(label_list[0])

1323
0


In [79]:
### Create sequence
first_1000 = df['sentence']
label_list = df['etype']
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(first_1000)

sequences = tokenizer.texts_to_sequences(first_1000)
data = pad_sequences(sequences, maxlen=200)

In [80]:
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = glove_300d.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label_list, test_size=0.33, random_state=42)

In [82]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 300, input_length=200, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(200))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [83]:
model_glove.fit(X_train, np.array(y_train), validation_split=0.0, epochs=10)

Epoch 1/10
886/886 [==============================] - 6s 7ms/step - loss: 0.5939 - acc: 0.7212
Epoch 2/10
886/886 [==============================] - 4s 5ms/step - loss: 0.4517 - acc: 0.7833
Epoch 3/10
886/886 [==============================] - 4s 5ms/step - loss: 0.3547 - acc: 0.8499
Epoch 4/10
886/886 [==============================] - 4s 5ms/step - loss: 0.2923 - acc: 0.8758
Epoch 5/10
886/886 [==============================] - 4s 5ms/step - loss: 0.2301 - acc: 0.8962
Epoch 6/10
886/886 [==============================] - 4s 5ms/step - loss: 0.1214 - acc: 0.9628
Epoch 7/10
886/886 [==============================] - 4s 5ms/step - loss: 0.0728 - acc: 0.9808
Epoch 8/10
886/886 [==============================] - 4s 5ms/step - loss: 0.0247 - acc: 0.9932
Epoch 9/10
886/886 [==============================] - 4s 5ms/step - loss: 0.0402 - acc: 0.9910
Epoch 10/10
886/886 [==============================] - 4s 5ms/step - loss: 0.0442 - acc: 0.9876


In [84]:
prediction = model_glove.predict(X_test)
prediction = [1 if pred>0.5 else 0 for pred in prediction]

In [85]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print(f1_score(y_test, prediction))
print(accuracy_score(y_test, prediction))
print(precision_score(y_test, prediction))
print(recall_score(y_test, prediction))

0.6925795053003533
0.8009153318077803
0.6533333333333333
0.7368421052631579
